In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.ML.Descriptors.Descriptors import DescriptorCalculator
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
import numpy as np
import catboost
from tqdm import tqdm
from sklearn import metrics
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from statistics import mean, stdev
from platform import python_version
print(python_version())


3.10.9


In [2]:
def read_data(file_name):
    df = pd.read_excel(file_name)
    df['SMILES'] = df['SMILES'].replace('>>', '.', regex=True)
    return df

In [3]:
def calculate_descriptors(df):
    descriptor_names = list(Chem.rdMolDescriptors.Properties.GetAvailableProperties())
    descriptor_names.remove('exactmw')
    for name in tqdm(descriptor_names):
        df[name] = df['SMILES'].apply(lambda x: descriptor(name, x))
    return df

In [4]:
def descriptor(descriptor:str, smi:str)->float:
    get_descriptors = Chem.rdMolDescriptors.Properties([descriptor])
    try:
        return get_descriptors.ComputeProperties(Chem.MolFromSmiles(smi))[0]
    except:
        return None

In [5]:
def filter_descriptors(data, threshold=0.9):
    descriptors = data.drop(["SMILES", 'yield', 'Ind'], axis=1)
    corr = descriptors.copy().corr()
    for index in corr.index:
        corr.loc[index, index] = 0
    descriptors_not_correlated = corr[(corr <= threshold)].dropna().index.tolist()
    print('Number of descriptors:', len(descriptors_not_correlated) - 1)
    return data[descriptors_not_correlated], data['yield']

In [6]:
def random_forest_regression(X, y):
    rmse_scores = []
    r2_scores = []
    for i in range(10):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
        model = RandomForestRegressor(n_estimators=800, max_depth=8)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        rmse_scores.append(rmse)
        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)
        #print(f"Iteration {i}: RMSE = {rmse:.3f}, R^2  = {r2:.3f}")
    print("\nAverage performance of test set at 10 split")
    print("Metric\tavg\tstdev")
    print("R^2\t%.2f\t%.2f" % (mean(r2_scores), stdev(r2_scores)))
    print("RMSE\t%.2f\t%.2f" % (mean(rmse_scores), stdev(rmse_scores)))

In [7]:
def statistics_for_the_training_set_with_5CV_RFR(X, y):
    r2_scores = []
    rmse_scores = []
    for i in range(10):
        X_tr, X_t, y_tr, y_t = train_test_split(X, y, test_size=0.2, random_state=i)
        X_tr.reset_index(drop=True , inplace=True)
        y_tr.reset_index(drop=True , inplace=True)
        cv = KFold(n_splits=5, shuffle=True, random_state=1)
        y_pred, y_true = [], []
        for train_index, test_index in cv.split(X_tr):
            X_train = X_tr.loc[train_index].values
            X_test = X_tr.loc[test_index].values
            y_train = y_tr.loc[train_index].values
            y_test = y_tr.loc[test_index].values
    
            model = RandomForestRegressor(n_estimators=800, max_depth=8)
            model.fit(X_train, y_train)
            y_pred.extend(model.predict(X_test))
            y_true.extend(y_test)

        r2_scores.append(metrics.r2_score(y_true, y_pred))
        rmse_scores.append(metrics.mean_squared_error(y_true, y_pred, squared=False))
      
    print("\nAverage performance of 5CV")
    print("Metric\tavg\tstdev")
    print("R^2\t%.2f\t%.2f" % (mean(r2_scores), stdev(r2_scores)))
    print("RMSE\t%.2f\t%.2f" % (mean(rmse_scores), stdev(rmse_scores)))  

In [8]:
def catboost_regression(X, y):
    rmse_scores = []
    r2_scores = []
    for i in range(10):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
        model = catboost.CatBoostRegressor(learning_rate=0.02, l2_leaf_reg=9.5, depth=7, silent=True)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        rmse_scores.append(rmse)
        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)
        #print(f"Iteration {i}: RMSE = {rmse:.3f}, R2 = {r2:.3f}")
    print("\nAverage performance of test set at 10 split")
    print("Metric\tavg\tstdev")
    print("R^2\t%.2f\t%.2f" % (mean(r2_scores), stdev(r2_scores)))
    print("RMSE\t%.2f\t%.2f" % (mean(rmse_scores), stdev(rmse_scores)))

In [9]:
def statistics_for_the_training_set_with_5CV_catBoost(X, y):
    r2_scores = []
    rmse_scores = []
    for i in range(10):
        X_tr, X_t, y_tr, y_t = train_test_split(X, y, test_size=0.2, random_state=i)
        X_tr.reset_index(drop=True , inplace=True)
        y_tr.reset_index(drop=True , inplace=True)
        cv = KFold(n_splits=5, shuffle=True, random_state=1)
        y_pred, y_true = [], []
        for train_index, test_index in cv.split(X_tr):
            X_train = X_tr.loc[train_index].values
            X_test = X_tr.loc[test_index].values
            y_train = y_tr.loc[train_index].values
            y_test = y_tr.loc[test_index].values
    
            model = catboost.CatBoostRegressor(learning_rate=0.02, l2_leaf_reg=9.5, depth=7, silent=True)
            model.fit(X_train, y_train)
            y_pred.extend(model.predict(X_test))
            y_true.extend(y_test)

        r2_scores.append(metrics.r2_score(y_true, y_pred))
        rmse_scores.append(metrics.mean_squared_error(y_true, y_pred, squared=False))
      
    print("\nAverage performance of 5CV")
    print("Metric\tavg\tstdev")
    print("R^2\t%.2f\t%.2f" % (mean(r2_scores), stdev(r2_scores)))
    print("RMSE\t%.2f\t%.2f" % (mean(rmse_scores), stdev(rmse_scores)))        

In [10]:
def product_strategy_random_forest(data, X, y):
    unique_inds = data['Ind'].unique()
    rmse_scores = []
    r2_scores = []
    for i in range(10):
        ind_train, ind_test = train_test_split(unique_inds, test_size=0.20, random_state=i)
        train_indices = data[data['Ind'].isin(ind_train)].index
        test_indices = data[data['Ind'].isin(ind_test)].index
        x_train, y_train = X.loc[train_indices].values, y.loc[train_indices].values
        x_test, y_test = X.loc[test_indices].values, y.loc[test_indices].values
        model = RandomForestRegressor(n_estimators=800, max_depth=8)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        rmse_scores.append(rmse)
        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)
        #print(f"Iteration {i}: RMSE = {rmse:.3f}, R2 = {r2:.3f}")
    print("\nAverage performance of test set at 10 split")
    print("Metric\tavg\tstdev")
    print("R^2\t%.2f\t%.2f" % (mean(r2_scores), stdev(r2_scores)))
    print("RMSE\t%.2f\t%.2f" % (mean(rmse_scores), stdev(rmse_scores)))

In [15]:
def product_statistics_for_the_training_set_with_5CV_RFR(data, X, y):
    unique_inds = data['Ind'].unique()
    ind_train, ind_test = train_test_split(unique_inds, test_size=0.2, random_state=1)

    train_indices = data[data['Ind'].isin(ind_train)].index
    test_indices = data[data['Ind'].isin(ind_test)].index

    x_train, y_train = X.loc[train_indices].values, y.loc[train_indices].values
    x_test, y_test = X.loc[test_indices].values, y.loc[test_indices].values

    kf = KFold(n_splits=5, shuffle=True, random_state=1)
    rmse_scores = []
    r2_scores = []

    for train_index, test_index in kf.split(x_train):
        x_train_fold, x_val_fold = x_train[train_index], x_train[test_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[test_index]
    
        model = RandomForestRegressor(n_estimators=800, max_depth=8, verbose=0)  # Use verbose=0 instead of silent=True
        model.fit(x_train_fold, y_train_fold)
        y_pred = model.predict(x_val_fold)
    
        rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
        rmse_scores.append(rmse)
    
        r2 = r2_score(y_val_fold, y_pred)
        r2_scores.append(r2)
    
    print('R^2 = ', round(np.mean(r2_scores), 2))
    print('RMSE = ', round(np.mean(rmse_scores), 2))


In [12]:
def product_strategy_catboost(data, X, y):
    unique_inds = data['Ind'].unique()
    rmse_scores = []
    r2_scores = []
    for i in range(10):
        ind_train, ind_test = train_test_split(unique_inds, test_size=0.20, random_state=i)
        train_indices = data[data['Ind'].isin(ind_train)].index
        test_indices = data[data['Ind'].isin(ind_test)].index
        x_train, y_train = X.loc[train_indices].values, y.loc[train_indices].values
        x_test, y_test = X.loc[test_indices].values, y.loc[test_indices].values
        model = catboost.CatBoostRegressor(learning_rate=0.02, l2_leaf_reg=9.5, depth=7, silent=True)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        rmse_scores.append(rmse)
        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)
        #print(f"Iteration {i}: RMSE = {rmse:.3f}, R2 = {r2:.3f}")
    print("\nAverage performance of test set at 10 split")
    print("Metric\tavg\tstdev")
    print("R^2\t%.2f\t%.2f" % (mean(r2_scores), stdev(r2_scores)))
    print("RMSE\t%.2f\t%.2f" % (mean(rmse_scores), stdev(rmse_scores)))

In [13]:
def product_statistics_for_the_training_set_with_5CV_catBoost(data,X, y):
    unique_inds = data['Ind'].unique()
    ind_train, ind_test = train_test_split(unique_inds, test_size=0.2, random_state=1)

    train_indices = data[data['Ind'].isin(ind_train)].index
    test_indices = data[data['Ind'].isin(ind_test)].index

    x_train, y_train = X.loc[train_indices].values, y.loc[train_indices].values
    x_test, y_test = X.loc[test_indices].values, y.loc[test_indices].values

    kf = KFold(n_splits=5, shuffle=True, random_state=1)
    rmse_scores = []
    r2_scores = []

    for train_index, test_index in kf.split(x_train):
        x_train_fold, x_val_fold = x_train[train_index], x_train[test_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[test_index]
    
        model = catboost.CatBoostRegressor(learning_rate=0.02, l2_leaf_reg=9.5, depth=7, silent=True)
        model.fit(x_train_fold, y_train_fold)
        y_pred = model.predict(x_val_fold)
    
        rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
        rmse_scores.append(rmse)
    
        r2 = r2_score(y_val_fold, y_pred)
        r2_scores.append(r2)
    
    print('R^2 = ', round(np.mean(r2_scores), 2))
    print('RMSE = ', round(np.mean(rmse_scores), 2))

In [16]:
if __name__ == "__main__":
    df = read_data("Condensation_reactions.xls")
    df = calculate_descriptors(df)
    data = df.dropna()
    data.reset_index(drop=True, inplace=True)
    X, y = filter_descriptors(data)

    # The "reaction" strategy
    print("Random Forest Regression:")
    random_forest_regression(X, y)

    print("\nStatistics for the training set with 5CV RFR:")
    statistics_for_the_training_set_with_5CV_RFR(X, y)
    
    print("\nCatBoost Regression:")
    catboost_regression(X, y)
    
    print("\nStatistics for the training set with 5CV catBoost:")
    statistics_for_the_training_set_with_5CV_catBoost(X, y)

    # The "product" strategy
    print("\nProduct Strategy with Random Forest Regression:")
    product_strategy_random_forest(data, X, y)

    print("\nProduct Strategy with Statistics for the training set with 5CV RFR:")
    product_statistics_for_the_training_set_with_5CV_RFR(data,X, y)
    
    print("\nProduct Strategy with CatBoost Regression:")
    product_strategy_catboost(data, X, y)
    
    print("\nProduct Strategy with Statistics for the training set with 5CV catBoost:")
    product_statistics_for_the_training_set_with_5CV_catBoost(data, X, y)


  0%|          | 0/42 [00:00<?, ?it/s][15:30:51] Explicit valence for atom # 26 H, 2, is greater than permitted
[15:30:51] Explicit valence for atom # 26 H, 2, is greater than permitted
[15:30:51] Explicit valence for atom # 0 O, 3, is greater than permitted
[15:30:51] Explicit valence for atom # 10 O, 3, is greater than permitted
  2%|▏         | 1/42 [00:00<00:16,  2.54it/s][15:30:51] Explicit valence for atom # 26 H, 2, is greater than permitted
[15:30:51] Explicit valence for atom # 26 H, 2, is greater than permitted
[15:30:51] Explicit valence for atom # 0 O, 3, is greater than permitted
[15:30:52] Explicit valence for atom # 10 O, 3, is greater than permitted
  5%|▍         | 2/42 [00:00<00:15,  2.58it/s][15:30:52] Explicit valence for atom # 26 H, 2, is greater than permitted
[15:30:52] Explicit valence for atom # 26 H, 2, is greater than permitted
[15:30:52] Explicit valence for atom # 0 O, 3, is greater than permitted
[15:30:52] Explicit valence for atom # 10 O, 3, is greater 

Number of descriptors: 13
Random Forest Regression:

Average performance of test set at 10 split
Metric	avg	stdev
R^2	0.56	0.05
RMSE	13.99	0.73

Statistics for the training set with 5CV RFR:

Average performance of 5CV
Metric	avg	stdev
R^2	0.55	0.02
RMSE	14.25	0.27

CatBoost Regression:

Average performance of test set at 10 split
Metric	avg	stdev
R^2	0.57	0.04
RMSE	13.87	0.66

Statistics for the training set with 5CV catBoost:

Average performance of 5CV
Metric	avg	stdev
R^2	0.56	0.02
RMSE	14.14	0.18

Product Strategy with Random Forest Regression:

Average performance of test set at 10 split
Metric	avg	stdev
R^2	0.54	0.06
RMSE	14.26	0.99

Product Strategy with Statistics for the training set with 5CV RFR:
R^2 =  0.57
RMSE =  14.14

Product Strategy with CatBoost Regression:

Average performance of test set at 10 split
Metric	avg	stdev
R^2	0.54	0.06
RMSE	14.18	1.05

Product Strategy with Statistics for the training set with 5CV catBoost:
R^2 =  0.56
RMSE =  14.2
